In [1]:
import gdal
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
from glob import glob
import shapely
import random
from tqdm import tqdm
from collections import Counter
from pybob.GeoImg import GeoImg
import osr
from pybob.image_tools import create_mask_from_shapefile
import gc
import imageio
import time
import shutil

In [2]:
def get_mask(maskshp,geotrf,shape,p='4326'):
    img=np.zeros((shape[0],shape[1]))
    
    if p=='4326':
        sref = osr.SpatialReference()
        sref.ImportFromEPSG(4326)
        proj = sref.ExportToWkt()
    else:
        proj=p
    
    trf=(geotrf[0], geotrf[1], geotrf[2], geotrf[3], geotrf[4], geotrf[5])
    drv = gdal.GetDriverByName('MEM')
    dst = drv.Create('', shape[1], shape[0], 1, gdal.GDT_UInt16)

    sp = dst.SetProjection(proj)
    sg = dst.SetGeoTransform(trf)

    wa = dst.GetRasterBand(1).WriteArray(img)
    del wa, sg, sp
    img_bob = GeoImg(dst)
    mask = create_mask_from_shapefile(img_bob, maskshp)
    return mask
def read_img(path):
    ds=gdal.Open(path)
    trf=ds.GetGeoTransform()
    img=ds.ReadAsArray()
    return {'trf':trf,'img':img}
def tif_save_bands(img,save_name,trf,p='4326'):
    driver=gdal.GetDriverByName('GTiff')
    new_img=driver.Create(save_name,img.shape[2],img.shape[1],img.shape[0],6,['COMPRESS=LZW','BIGTIFF=YES'])
    new_img.SetGeoTransform(trf)
    
    if p=='4326':
        sref = osr.SpatialReference()
        sref.ImportFromEPSG(4326)
        proj = sref.ExportToWkt()
    else:
        proj=p
    new_img.SetProjection(proj)
    for i in range(img.shape[0]):
        new_img.GetRasterBand(i+1).WriteArray(img[i,:,:])
    new_img.FlushCache()
    del new_img
def tif_save(img,save_name,trf,p='4326'):
    driver=gdal.GetDriverByName('GTiff')
    new_img=driver.Create(save_name,img.shape[1],img.shape[0],1,6,['COMPRESS=LZW','BIGTIFF=YES'])
    new_img.SetGeoTransform(trf)
    
    if p=='4326':
        sref = osr.SpatialReference()
        sref.ImportFromEPSG(4326)
        proj = sref.ExportToWkt()
    else:
        proj=p
    new_img.SetProjection(proj)
    new_img.GetRasterBand(1).SetNoDataValue(0)
    new_img.GetRasterBand(1).WriteArray(img)
    new_img.FlushCache()
    del new_img

# 图像预处理

In [8]:
cities=['Homs', 'Idlib', 'Raqqa', 'Aleppo', 'Deir-ez-Zor', 'Hama']
pathes={'Homs':['/data4/Syria_img/Homs_A_20100629_treated.tif',
 '/data4/Syria_img/Homs_enhance-A-B-1_20120531_treated.tif',
 '/data4/Syria_img/Homs_enhance-A-B-2_20120829_treated.tif',
 '/data4/Syria_img/Homs_enhance-A-B-3_20130309_treated.tif',
 '/data4/Syria_img/Homs_B_20130926_treated.tif',
 '/data4/Syria_img/Homs_enhance-B-C-1_20131031_treated.tif',
 '/data4/Syria_img/Homs_enhance-B-C-2_20140403_treated.tif',
 '/data4/Syria_img/Homs_C_20140421_treated.tif',
 '/data4/Syria_img/Homs_enhance-C-Z-1_20140609_treated.tif',
 '/data4/Syria_img/Homs_enhance-C-Z-2_20160530_treated.tif',
 '/data4/Syria_img/Homs_enhance-C-Z-3_20160601_treated.tif',
 '/data4/Syria_img/Homs_rebuilding-1_20200508_treated.tif',
 '/data4/Syria_img/Homs_rebuilding-2_20201228_treated.tif',
 '/data4/Syria_img/Homs_rebuilding-3_20210629_treated.tif'],
 'Idlib':['/data4/Syria_img/Idlib_A_20100322_treated.tif',
 '/data4/Syria_img/Idlib_enhance-A-B-1_20120927_treated.tif',
 '/data4/Syria_img/Idlib_enhance-A-B-2_20121003_treated.tif',
 '/data4/Syria_img/Idlib_enhance-A-B-3_20130824_treated.tif',
 '/data4/Syria_img/Idlib_B_20130915_treated.tif',
 '/data4/Syria_img/Idlib_enhance-B-C-1_20140207_treated.tif',
 '/data4/Syria_img/Idlib_C_20140502_treated.tif',
 '/data4/Syria_img/Idlib_enhance-C-D-1_20140531_treated.tif',
 '/data4/Syria_img/Idlib_enhance-C-D-2_20140924_treated.tif',
 '/data4/Syria_img/Idlib_D_20150406_treated.tif',
 '/data4/Syria_img/Idlib_enhance-D-E-1_20150417_treated.tif',
 '/data4/Syria_img/Idlib_enhance-D-E-3_20160601_treated.tif',
 '/data4/Syria_img/Idlib_E_20160801_treated.tif',
 '/data4/Syria_img/Idlib_enhance-E-Z-1_20161001_treated.tif',
 '/data4/Syria_img/Idlib_enhance-E-Z-2_20170221_treated.tif',
 '/data4/Syria_img/Idlib_enhance-E-Z-3_20170717_treated.tif',
 '/data4/Syria_img/Idlib_rebuilding-1_20201023_treated.tif',
 '/data4/Syria_img/Idlib_rebuilding-2_20201227_treated.tif',
 '/data4/Syria_img/Idlib_rebuilding-3_20210819_treated.tif'],
 'Raqqa':['/data4/Syria_img/Raqqa_A_20100703_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-1_20120928_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-2_20121010_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-3_20130117_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-4_20130213_treated.tif',
 '/data4/Syria_img/Raqqa_C_20140212_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-C-D-1_20140321_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-C-D-2_20141006_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-C-D-3_20150202_treated.tif',
 '/data4/Syria_img/Raqqa_D_20150529_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-1_20160220_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-2_20160613_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-3_20160916_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-4_20170701_treated.tif',
  '/data4/Syria_img/Raqqa_rebuilding-1_20200703_treated.tif'],
 'Aleppo':['/data4/Syria_img/Aleppo_A_20100704_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-A-B-1_20121230_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-A-B-2_20130226_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-A-B-3_20130526_treated.tif',
 '/data4/Syria_img/Aleppo_B_20130923_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-B-C-1_20131031_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-B-C-2_20140131_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-B-C-3_20140426_treated.tif',
 '/data4/Syria_img/Aleppo_C_20140523_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-C-D-1_20140714_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-C-D-2_20140918_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-C-D-3_20141215_treated.tif',
 '/data4/Syria_img/Aleppo_D_20150426_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-D-E-1_20151122_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-D-E-2_20160425_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-D-E-3_20160803_treated.tif',
 '/data4/Syria_img/Aleppo_E_20150918_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-E-Z-1_20160925_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-E-Z-2_20161001_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-E-Z-3_20161010_treated.tif',
  '/data4/Syria_img/Aleppo_rebuilding-1_20210110_treated.tif',
 '/data4/Syria_img/Aleppo_rebuilding-2_20210327_treated.tif',
 '/data4/Syria_img/Aleppo_rebuilding-3_20210730_treated.tif'],
 'Deir-ez-Zor': ['/data4/Syria_img/Deir-ez-Zor_A_20090327_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-A-B-1_20121205_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-A-B-2_20130213_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-A-B-3_20130223_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_B_20131024_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_C_20140513_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-C-D-1_20140916_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-C-D-2_20141015_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-C-D-3_20141125_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_D_20150510_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-D-E-1_20160417_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_E_20160525_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-E-Z-1_20160706_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-E-Z-2_20160813_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-E-Z-3_20160913_treated.tif',
  '/data4/Syria_img/Deir-ez-Zor_rebuilding-1_20210717_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_rebuilding-2_20210818_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_rebuilding-3_20210829_treated.tif'],
 'Hama':['/data4/Syria_img/Hama_A_20090607_treated.tif',
 '/data4/Syria_img/Hama_enhance-A-B-1_20120921_treated.tif',
 '/data4/Syria_img/Hama_enhance-A-B-2_20130309_treated.tif',
 '/data4/Syria_img/Hama_enhance-A-B-3_20130827_treated.tif',
 '/data4/Syria_img/Hama_B_20130926_treated.tif',
 '/data4/Syria_img/Hama_enhance-B-C-1_20131031_treated.tif',
 '/data4/Syria_img/Hama_C_20140305_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-1_20140526_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-2_20140613_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-3_20140804_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-4_20140917_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-5_20160518_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-6_20160530_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-1_20160913_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-2_20161010_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-3_20170221_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-4_20170730_treated.tif',
  '/data4/Syria_img/Hama_rebuilding-1_20200626_treated.tif',
 '/data4/Syria_img/Hama_rebuilding-2_20200819_treated.tif',
 '/data4/Syria_img/Hama_rebuilding-3_20210812_treated.tif']}



In [11]:
pathes_norebuilding={}
for city in cities:
    p=pathes[city]
    pathes_norebuilding[city]=[i for i in p if 'rebuilding' not in i]
pathes_norebuilding

{'Homs': ['/data4/Syria_img/Homs_A_20100629_treated.tif',
  '/data4/Syria_img/Homs_enhance-A-B-1_20120531_treated.tif',
  '/data4/Syria_img/Homs_enhance-A-B-2_20120829_treated.tif',
  '/data4/Syria_img/Homs_enhance-A-B-3_20130309_treated.tif',
  '/data4/Syria_img/Homs_B_20130926_treated.tif',
  '/data4/Syria_img/Homs_enhance-B-C-1_20131031_treated.tif',
  '/data4/Syria_img/Homs_enhance-B-C-2_20140403_treated.tif',
  '/data4/Syria_img/Homs_C_20140421_treated.tif',
  '/data4/Syria_img/Homs_enhance-C-Z-1_20140609_treated.tif',
  '/data4/Syria_img/Homs_enhance-C-Z-2_20160530_treated.tif',
  '/data4/Syria_img/Homs_enhance-C-Z-3_20160601_treated.tif'],
 'Idlib': ['/data4/Syria_img/Idlib_A_20100322_treated.tif',
  '/data4/Syria_img/Idlib_enhance-A-B-1_20120927_treated.tif',
  '/data4/Syria_img/Idlib_enhance-A-B-2_20121003_treated.tif',
  '/data4/Syria_img/Idlib_enhance-A-B-3_20130824_treated.tif',
  '/data4/Syria_img/Idlib_B_20130915_treated.tif',
  '/data4/Syria_img/Idlib_enhance-B-C-1_20140

In [18]:
for city in cities:
    ps=pathes_norebuilding[city]
    for p in ps:
        if not os.path.exists(p.replace('.tif',f'_size_120.npy')):
            print(p)

In [31]:
import torch
import geopandas as gpd
import numpy as np
from tqdm import tqdm
import imageio
liveuamap_point=gpd.read_file('/data4/liveuamap/S_point_use_bomb_xy.shp')
for city in cities:
    print(city)
    city_gpd=liveuamap_point[liveuamap_point['region']==city]
    x=list(city_gpd['patch_x'])
    y=list(city_gpd['patch_y'])
    coords=[[y[i],x[i]] for i in range(len(x))]
    ps=pathes_norebuilding[city]
    pre_patch=np.load(ps[0].replace('.tif',f'_size_120.npy'))
    infos={}
    save_root_patch='/ssd/hk/Syria_samples/liveuamap_samples/'
    for i in range(1,len(ps)):
        print(ps[i])
        img_id=os.path.basename(ps[i]).split('_')[1]
        post_patch=np.load(ps[i].replace('.tif',f'_size_120.npy'))
        infos_p={}
        for coord in tqdm(coords):
            c_key=str(coord[0]).zfill(3)+str(coord[1]).zfill(3)
            pre_path=save_root_patch+'pre/'+f'{city}_{img_id}_{c_key}.png'
            post_path=pre_path.replace('/pre/','/post/')
            infos_p[c_key]=[pre_path,post_path]
            imageio.imwrite(pre_path,pre_patch[:,coord[0],coord[1]].transpose(1,2,0).astype(np.uint8))
            imageio.imwrite(post_path,post_patch[:,coord[0],coord[1]].transpose(1,2,0).astype(np.uint8))
        infos[ps[i]]=infos_p
    torch.save(infos,save_root_patch+f'{city}_samples_info.pth')

Homs
/data4/Syria_img/Homs_enhance-A-B-1_20120531_treated.tif


100%|██████████| 5/5 [00:00<00:00, 109.90it/s]


/data4/Syria_img/Homs_enhance-A-B-2_20120829_treated.tif


100%|██████████| 5/5 [00:00<00:00, 77.83it/s]


/data4/Syria_img/Homs_enhance-A-B-3_20130309_treated.tif


...

100%|██████████| 3/3 [00:00<00:00, 67.87it/s]


/data4/Syria_img/Hama_enhance-D-Z-3_20170221_treated.tif


100%|██████████| 3/3 [00:00<00:00, 94.06it/s]


/data4/Syria_img/Hama_enhance-D-Z-4_20170730_treated.tif


100%|██████████| 3/3 [00:00<00:00, 101.79it/s]


In [97]:
zzz=[]
for i in range(len(liveuamap_point)):
    p=liveuamap_point.iloc[i]
    c_key=str(p['patch_y']).zfill(3)+str(p['patch_x']).zfill(3)
    city=p['region']
    sample_mt_path=[]
    for i in pathes_norebuilding[city]:
        img_id=os.path.basename(i).split('_')[1]
        if img_id=='A':
            continue
        else:
            sample_path=f'/ssd/hk/Syria_samples/liveuamap_samples/post/{city}_{img_id}_{c_key}.png'
            if not os.path.exists(sample_path):
                print(sample_path)
                raise()
            else:
                sample_mt_path.append(sample_path)
    zzz.append(sample_mt_path)
zzz=[i if len(i)<17 else i[-16:] for i in zzz]
kkk=[]
for kk in zzz:
    k=''
    for i in kk:
        k+='&'
        k+=i
    k=k[1:]
    kkk.append(k)
liveuamap_point['img_path']=kkk

In [100]:
dates={}
for city in pathes_norebuilding.keys():
    ps=pathes_norebuilding[city][1:]
    dates_city=[]
    for p in ps:
        dates_city.append(os.path.basename(p).split('_')[2])
    if len(dates_city)>16:
        dates_city=dates_city[-16:]
    dates_city_str=''
    for i in dates_city:
        dates_city_str+='&'
        dates_city_str+=i
    
    dates[city]=dates_city_str[1:]
dd=[]
for i in range(len(liveuamap_point)):
    p=liveuamap_point.iloc[i]
    city=p['region']
    dd.append(dates[city])
liveuamap_point['img_dates']=dd
        
# liveuamap_point.to_file('/data4/liveuamap/S_point_use_bomb_xy_withimg.shp')


In [27]:
import sys
import imageio
import matplotlib.pyplot as plt
import torch
import numpy as np
sys.path.append('..')
from model import mynet6size120mtc1v7
from model import mynet6size120
from model import resnet50
# model=mynet6size120mtc1v7.mynet6size120mtc1v7()
from collections import OrderedDict
def cat(img_pre,img_post):
    data=np.concatenate((img_pre,img_post),axis=0)
    data=torch.Tensor(data)
    return data
def get_pre_result(path_post):
    w=torch.load('/home/hk/python_script/Ukraine/checkpoints/试一试_mynet6size120mtc1v7_Syria_size120_allcitiesall_effectweight_v3_multi-time_RandomDelandCopyv2_6/best_ckpt.pt')['model_state_dict']
    w2={}
    for k in w.keys():

        k2=k.replace('module.','')
        w2[k2]=w[k]
    w2=OrderedDict(w2)
    model=mynet6size120mtc1v7.mynet6size120mtc1v7()
    model.load_state_dict(w2)
    labels_v=[1]*len(path_post)
    zeros_img_len=16-len(path_post)
    path_post+=[None]*zeros_img_len
    labels_v+=[-1]*zeros_img_len
    path_pre=path_post[0].replace('/post/','/pre/')
    img_pre=imageio.imread(path_pre).transpose(2,0,1)
    imgs=[cat(img_pre,imageio.imread(post).transpose(2,0,1)) if post is not None else np.zeros((6,120,120)) for post in path_post]
    data=np.concatenate(imgs,axis=0)
    labels_v=torch.tensor(labels_v)
    input=[data,labels_v]
    input[0]=torch.Tensor(input[0])
    output=model(torch.Tensor(input[0]).unsqueeze(0))
    # print('feature1 shape:',feature1.shape)
    # print('feature2 shape:',feature2.shape)
    ll=input[1].cpu().numpy()
    pp=torch.argmax(output,dim=2)[0].cpu().numpy()
    pre_result=torch.argmax(output,dim=2)[0].cpu().numpy()[:16-zeros_img_len]
    softmax_0 = nn.Softmax(dim=1)
    output_softmax=softmax_0(output[0])[:,1].cpu().detach().numpy()[:16-zeros_img_len]
    return pre_result,output_softmax
pre_results=[]
pre_results_softmax=[]
for i in tqdm(range(len(liveuamap_point))):
    path_post=liveuamap_point.iloc[i]['img_path'].split('&')
    out,out_sm=get_pre_result(path_post)
    pre_results.append(out)
    pre_results_softmax.append(out_sm)

In [169]:
def to_str(l):
    ll=[]
    for i in l:
        ii=list(i)
        k=''
        for j in ii:
            k+='&'
            k+=str(j)
        ll.append(k[1:])
    return ll
liveuamap_point['MTCF-PtNet']=to_str(pre_results)
liveuamap_point['MTCF-PtNet_softmax']=to_str(pre_results_softmax)
liveuamap_point.to_file('/data4/liveuamap/S_point_use_bomb_xy_withimg.shp')

/home/hk/miniconda3/envs/pythonhzy/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  del sys.path[0]


In [204]:
import time
def to_timestamp(t):
    if not len(t)==8:
        raise('time length error')
    t=f'{t[:4]}-{t[4:6]}-{t[6:]}'
    struct_time=time.strptime(t,'%Y-%m-%d')
    return time.mktime(struct_time)

def get_before_after_prediction(loc):
    dates=loc['img_dates'].split('&')
    dates_stamp=[to_timestamp(i) for i in dates]
    event_t=loc['timestamp']
    prediction_c=[np.int(i) for i in loc['MTCF-PtNet'].split('&')]
    prediction_s=[np.float(i) for i in loc['MTCF-PtNet_softmax'].split('&')]
    prediction_c+=[-99]*10
    prediction_s+=[-99]*10
    for i in range(len(dates_stamp)):
        if dates_stamp[i]>event_t:
            break
    before_prediction_c=prediction_c[:i][-5:]
    before_prediction_s=prediction_s[:i][-5:]
    after_prediction_c=prediction_c[i:][:5]
    after_prediction_s=prediction_s[i:][:5]
    return before_prediction_c,before_prediction_s,after_prediction_c,after_prediction_s
# loc=liveuamap_point.iloc[100]
bpc,bps,apc,aps=[],[],[],[]
for i in range(len(liveuamap_point)):
    loc=liveuamap_point.iloc[i]
    a,b,c,d=get_before_after_prediction(loc)
    bpc.append(a)
    bps.append(b)
    apc.append(c)
    aps.append(d)
liveuamap_point['MTCF-PtNet_before']=to_str(bpc)
liveuamap_point['MTCF-PtNet_softmax_before']=to_str(bps)
liveuamap_point['MTCF-PtNet_after']=to_str(apc)
liveuamap_point['MTCF-PtNet_softmax_after']=to_str(aps)
# liveuamap_point
# liveuamap_point.to_file('/data4/liveuamap/S_point_use_bomb_xy_withresult.shp')



In [216]:
pathes=[liveuamap_point.iloc[i]['img_path'].split('&') for i in range(len(liveuamap_point))]
pathes2=[]
for i in pathes:
    pathes2+=i
pathes2=[[i,'n'] for i in pathes2]
torch.save(pathes2,'/ssd/hk/Syria_samples/liveuamap_samples/reference_v2.pth')

In [223]:
liveuamap_point['intervention']=[1]*len(liveuamap_point)

# 无干预样本

In [15]:

import random
import numpy as np
import geopandas as gpd
liveuamap_point=gpd.read_file('/data4/liveuamap/S_point_use_bomb_xy_withimg.shp')
s_all=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/train.pth')
s_c={}
for c in cities:
    s_c[c]=[]
for i in s_all:
    c=os.path.basename(i[0][0]).split('_')[0]
    l=[j for j in i[1] if j is not None]
    l=[1 if j=='p' else 0 for j in l]
    if np.array(l).sum()==0:
        s_c[c].append(i[0])
use_nointervention=[]
for c in cities:
    length=len(liveuamap_point[liveuamap_point['region']==c])
    s_cc=s_c[c]
    random.shuffle(s_cc)
    use_nointervention+=s_cc[:length*10]

In [20]:
from shapely.geometry import Point
import shapely
from glob import glob
k=[[os.path.basename(i[0]).split('_')[0],os.path.basename(i[0]).split('_')[-1][:6]] for i in use_nointervention]
region=[i[0] for i in k]
x=[int(i[1][3:]) for i in k]
y=[int(i[1][:3]) for i in k]
ids=[i*(-1) for i in range(len(k))]
PP=[]
for i in range(len(x)):
    city=region[i]
    fs=glob(f'/data4/Syria_img/{city}*pth')
    fs.sort()
    # print(fs[0])
    a=torch.load(fs[0])
    trf=a['trf']
    PP.append(Point([(x[i]+0.5)*trf[1]+trf[0],(y[i]+0.5)*trf[5]+trf[3]]))
lon=[shapely.geometry.mapping(i)['coordinates'][0] for i in PP]
lat=[shapely.geometry.mapping(i)['coordinates'][1] for i in PP]
nointervention={}
nointervention['id']=ids
nointervention['time']=['none']*len(k)
nointervention['timestamp']=['none']*len(k)
nointervention['type']=['none']*len(k)
nointervention['name']=['none']*len(k)
nointervention['is_use']=[1]*len(k)
nointervention['region']=region
nointervention['lat']=lat
nointervention['lon']=lon
nointervention['patch_x']=x
nointervention['patch_y']=y
nointervention['geometry']=PP

nointervention['intervention']=[0]*len(liveuamap_point)*10
nointervention_pd=gpd.GeoDataFrame.from_dict(nointervention)
nointervention_pd['MTCF-PtNet_before']=['none']*len(nointervention_pd)
nointervention_pd['MTCF-PtNet_softmax_before']=['none']*len(nointervention_pd)
nointervention_pd['MTCF-PtNet_after']=['none']*len(nointervention_pd)
nointervention_pd['MTCF-PtNet_softmax_after']=['none']*len(nointervention_pd)

In [23]:
from tqdm import tqdm
import imageio
for city in cities:
    print(city)
    city_gpd=nointervention_pd[nointervention_pd['region']==city]
    x=list(city_gpd['patch_x'])
    y=list(city_gpd['patch_y'])
    coords=[[y[i],x[i]] for i in range(len(x))]
    ps=pathes_norebuilding[city]
    pre_patch=np.load(ps[0].replace('.tif',f'_size_120.npy'))
    infos={}
    save_root_patch='/ssd/hk/Syria_samples/liveuamap_samples/'
    for i in range(1,len(ps)):
        print(ps[i])
        img_id=os.path.basename(ps[i]).split('_')[1]
        post_patch=np.load(ps[i].replace('.tif',f'_size_120.npy'))
        infos_p={}
        for coord in tqdm(coords):
            c_key=str(coord[0]).zfill(3)+str(coord[1]).zfill(3)
            pre_path=save_root_patch+'pre/'+f'{city}_{img_id}_{c_key}.png'
            post_path=pre_path.replace('/pre/','/post/')
            infos_p[c_key]=[pre_path,post_path]
            imageio.imwrite(pre_path,pre_patch[:,coord[0],coord[1]].transpose(1,2,0).astype(np.uint8))
            imageio.imwrite(post_path,post_patch[:,coord[0],coord[1]].transpose(1,2,0).astype(np.uint8))
        infos[ps[i]]=infos_p
    torch.save(infos,save_root_patch+f'{city}_samples_info_nointervention.pth')

Homs
/data4/Syria_img/Homs_enhance-A-B-1_20120531_treated.tif


100%|██████████| 50/50 [00:00<00:00, 111.79it/s]


/data4/Syria_img/Homs_enhance-A-B-2_20120829_treated.tif


100%|██████████| 50/50 [00:00<00:00, 99.15it/s] 


/data4/Syria_img/Homs_enhance-A-B-3_20130309_treated.tif


...

100%|██████████| 30/30 [00:00<00:00, 70.59it/s]


/data4/Syria_img/Hama_enhance-D-Z-3_20170221_treated.tif


100%|██████████| 30/30 [00:00<00:00, 93.28it/s]


/data4/Syria_img/Hama_enhance-D-Z-4_20170730_treated.tif


100%|██████████| 30/30 [00:00<00:00, 103.93it/s]


In [25]:
zzz=[]
for i in range(len(nointervention_pd)):
    p=nointervention_pd.iloc[i]
    c_key=str(p['patch_y']).zfill(3)+str(p['patch_x']).zfill(3)
    city=p['region']
    sample_mt_path=[]
    for i in pathes_norebuilding[city]:
        img_id=os.path.basename(i).split('_')[1]
        if img_id=='A':
            continue
        else:
            sample_path=f'/ssd/hk/Syria_samples/liveuamap_samples/post/{city}_{img_id}_{c_key}.png'
            if not os.path.exists(sample_path):
                print(sample_path)
                raise()
            else:
                sample_mt_path.append(sample_path)
    zzz.append(sample_mt_path)
zzz=[i if len(i)<17 else i[-16:] for i in zzz]
kkk=[]
for kk in zzz:
    k=''
    for i in kk:
        k+='&'
        k+=i
    k=k[1:]
    kkk.append(k)
nointervention_pd['img_path']=kkk

In [31]:
from torch import nn

In [32]:
pre_results=[]
pre_results_softmax=[]
for i in tqdm(range(len(nointervention_pd))):
    path_post=nointervention_pd.iloc[i]['img_path'].split('&')
    out,out_sm=get_pre_result(path_post)
    pre_results.append(out)
    pre_results_softmax.append(out_sm)

100%|██████████| 10530/10530 [1:05:24<00:00,  2.68it/s]


In [39]:
def to_str(l):
    ll=[]
    for i in l:
        ii=list(i)
        k=''
        for j in ii:
            k+='&'
            k+=str(j)
        ll.append(k[1:])
    return ll
nointervention_pd['MTCF-PtNet']=to_str(pre_results)
nointervention_pd['MTCF-PtNet-softmax']=to_str(pre_results_softmax)
nointervention_pd.to_file('/data4/liveuamap/S_point_use_bomb_xy_withimg_no.shp')

/home/hk/miniconda3/envs/pythonhzy/lib/python3.7/site-packages/ipykernel_launcher.py:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  del sys.path[0]


In [50]:
a=[i.split('&') for i in nointervention_pd['MTCF-PtNet']]
b=[]
for i in a:
    b.append(np.array([int(j) for j in i]).sum())
nointervention_pd['b']=b
nointervention_pd=nointervention_pd[nointervention_pd['b']==0]
nointervention_pd['intervention']=[0]*len(nointervention_pd)

/home/hk/miniconda3/envs/pythonhzy/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [97]:
liveuamap_point=gpd.read_file('/data4/liveuamap/S_point_use_bomb_xy_withresult.shp')

del liveuamap_point['MTCF-PtN_2']
del liveuamap_point['MTCF-PtN_3']
del liveuamap_point['MTCF-PtN_4']
del liveuamap_point['MTCF-PtN_5']
liveuamap_point=liveuamap_point.rename(columns={'MTCF-PtN_1':'MTCF-PtNet-softmax'})

In [81]:
dates={}
import pandas as pd
for city in pathes_norebuilding.keys():
    ps=pathes_norebuilding[city][1:]
    dates_city=[]
    for p in ps:
        dates_city.append(os.path.basename(p).split('_')[2])
    if len(dates_city)>16:
        dates_city=dates_city[-16:]
    dates_city_str=''
    for i in dates_city:
        dates_city_str+='&'
        dates_city_str+=i
    
    dates[city]=dates_city_str[1:]
dd=[]
for i in range(len(nointervention_pd)):
    p=nointervention_pd.iloc[i]
    city=p['region']
    dd.append(dates[city])
nointervention_pd['img_dates']=dd

In [86]:
del nointervention_pd['b']
del nointervention_pd['MTCF-PtNet_before']
del nointervention_pd['MTCF-PtNet_after']
del nointervention_pd['MTCF-PtNet_softmax_before']
del nointervention_pd['MTCF-PtNet_softmax_after']

In [98]:
liveuamap_point['intervention']=[1]*len(liveuamap_point)

In [99]:
all_liveuamap=pd.concat([liveuamap_point,nointervention_pd])

In [121]:
all_liveuamap=all_liveuamap.rename(columns={'MTCF-softmax':'MTCF-sm','intervention':'interv'})

In [123]:
all_liveuamap.to_file('/data4/liveuamap/S_point_use_bomb_xy_withimg_all.shp')

In [461]:
a=[i.split('&') for i in list(all_liveuamap['MTCF-PtNet'])]
b=[i.split('&') for i in list(all_liveuamap['MTCF-PtNet_softmax'])]
c=[]
for i in range(len(a)):
    c.append(b[i][:len(a[i])])
all_liveuamap['MTCF-PtNet_softmax']=to_str(c)

In [129]:
liveuamap_point

,id,time,timestamp,type,name,is_use,region,lat,lon,patch_x,patch_y,img_path,img_dates,MTCF-PtNet,MTCF-PtNet-softmax,geometry,intervention
0,21199999,2014-12-27 17:40:00,1419673200,bomb-1,#Assad regime artillery targets a residential ...,1,Homs,34.73083,36.70944,68,55,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120531&20120829&20130309&20130926&20131031&2...,0&0&0&0&0&0&0&0&1&1,1.724617e-05&0.00023428141&7.843362e-05&9.1202...,POINT (36.70944 34.73083),1
1,21321405,2016-05-31 23:08:00,1464707280,bomb-1,Airstrikes on Lataminah in Hama,1,Homs,34.73243,36.71370,74,52,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120531&20120829&20130309&20130926&20131031&2...,0&0&0&0&0&1&1&1&1&1,1.3257952e-05&0.00037756359&0.00017163534&0.00...,POINT (36.71370 34.73243),1
2,21405736,2017-02-17 18:40:00,1487328000,bomb-1,Video from Al Waer district of Homs as Russian...,1,Homs,34.73243,36.71370,74,52,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120531&20120829&20130309&20130926&20131031&2...,0&0&0&0&0&1&1&1&1&1,1.3257952e-05&0.00037756359&0.00017163534&0.00...,POINT (36.71370 34.73243),1
3,21243220,2015-01-23 21:10:00,1422018600,bomb-2,The revolutionary forces targeted the regime t...,1,Homs,34.73075,36.70936,68,55,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120531&20120829&20130309&20130926&20131031&2...,0&0&0&0&0&0&0&0&1&1,1.724617e-05&0.00023428141&7.843362e-05&9.1202...,POINT (36.70936 34.73075),1
4,21243064,2015-01-22 19:36:00,1421926560,bomb-2,The FSA targeted the regime troops checkpoints...,1,Homs,34.73043,36.71082,70,55,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120531&20120829&20130309&20130926&20131031&2...,0&0&0&0&0&0&1&1&1&1,2.4961686e-05&0.00013824353&1.708659e-05&2.931...,POINT (36.71082 34.73043),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1048,21465749,2017-05-30 01:03:00,1496077380,bomb-3,Many civilians wounded in Qosour neighbourhood...,1,Deir-ez-Zor,35.33900,40.12761,72,38,/ssd/hk/Syria_samples/liveuamap_samples/post/D...,20121205&20130213&20130223&20131024&20140513&2...,0&0&0&0&0&0&0&0&0&0&1&1&1&1,0.15090048&0.0022120294&0.0018230478&0.0015210...,POINT (40.12761 35.33900),1
1049,21544911,2017-10-03 03:45:00,1506973500,bomb-3,ISIS targets safe neighborhoods in Deir Ezzor ...,1,Deir-ez-Zor,35.33515,40.12928,75,44,/ssd/hk/Syria_samples/liveuamap_samples/post/D...,20121205&20130213&20130223&20131024&20140513&2...,0&0&0&0&0&0&0&1&0&0&1&1&1&0,0.021988893&0.0011961802&0.0007855611&0.001439...,POINT (40.12928 35.33515),1
1050,21247599,2015-03-02 21:32:00,1425303120,bomb-1,"Assassination in Damascus city, and aerial bom...",1,Hama,35.13360,36.74965,51,77,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120921&20130309&20130827&20130926&20131031&2...,0&0&0&0&0&0&0&0&0&0&1&1&1&1&1&1,0.0004143768&0.00011727475&1.0972935e-05&0.000...,POINT (36.74965 35.13360),1
1051,21252215,2015-04-09 20:17:00,1428581820,bomb-1,Explosive barrels targeted the central provinc...,1,Hama,35.13325,36.74987,52,78,/ssd/hk/Syria_samples/liveuamap_samples/post/H...,20120921&20130309&20130827&20130926&20131031&2...,0&0&0&0&0&0&0&0&0&0&1&1&1&1&1&1,0.00018626731&8.0249614e-05&7.2216567e-06&0.00...,POINT (36.74987 35.13325),1


In [138]:
len(rsc)

10

In [139]:
import time
def to_timestamp(t):
    if not len(t)==8:
        raise('time length error')
    t=f'{t[:4]}-{t[4:6]}-{t[6:]}'
    struct_time=time.strptime(t,'%Y-%m-%d')
    return time.mktime(struct_time)
def get_res(event,ts):
    for i in range(len(ts)):
        if ts[i]>event:
            break
    return [i*(-1) for i in list(range(1,len(ts[:i])+1)[::-1])]+list(range(1,len(ts[i:])+1))
did_pd_list=[]
for i in range(len(all_liveuamap)):
    s=all_liveuamap.iloc[i]
    # rs=[np.float(i) for i in s['MTCF-sm'].split('&')]
    rsc=[np.int(i) for i in s['MTCF-PtNet'].split('&')]
    regions=[cities.index(s['region'])]*len(rsc)
    events=[s['timestamp']]*len(rsc)
    ids=[s['id']]*len(rsc)
    intervs=[s['interv']]*len(rsc)
    ts=[to_timestamp(i) for i in s['img_dates'].split('&')]
    if intervs[0]==0:
        res=[-99]*len(rsc)
    else:
        res=get_res(events[0],ts)
    did_pd_list.append(pd.DataFrame.from_dict({'id':ids,
                                                'time':ts,
                                                'interv_time':events,
                                                'period_difference':res,
                                                # 'value':rs,
                                                'value_c':rsc,
                                                'is_interv':intervs,
                                                'region':regions
                                                }))

In [240]:
did_pd=pd.concat(did_pd_list)

In [250]:
ps=[]
noexists=[]
zzzzz=[]
for i in range(len(liveuamap_point)):
    p=liveuamap_point.iloc[i]
    c_key=str(p['patch_y']).zfill(3)+str(p['patch_x']).zfill(3)
    city=p['region']
    try:
        pathes=[i for i in use_patch_dict[city][c_key][0] if i is not None]
        zzzzz.append(use_patch_dict[city][c_key][1])
        ps.append(pathes)
    except:
        noexists.append(p['id'])

for i in noexists:
    liveuamap_point=liveuamap_point.drop(liveuamap_point[liveuamap_point['id']==i].index)

In [247]:
len(noexists)

0

In [227]:
ts=[]
for i in ps:
    c=os.path.basename(i[0]).split('_')[0]
    tt=[]
    for j in i:
        k=os.path.basename(j).split('_')[1]
        t_p=glob(f'/data4/Syria_img/{c}_{k}*_treated.tif')[0]
        t=os.path.basename(t_p).split('_')[2]
        tt.append(t)
    ts.append(tt)

In [229]:
liveuamap_point['img_path']=to_str(ps)

In [231]:
liveuamap_point['img_dates']=to_str(ts)

In [237]:
pre_results_softmax[0]

array([3.83616025e-05, 7.58946317e-06, 1.19903576e-04, 2.70611454e-05,
       6.68417852e-05, 8.07432400e-04, 1.06878777e-03, 3.48833017e-03],
      dtype=float32)

In [232]:
pre_results=[]
pre_results_softmax=[]
for i in tqdm(range(len(liveuamap_point))):
    path_post=liveuamap_point.iloc[i]['img_path'].split('&')
    out,out_sm=get_pre_result(path_post)
    pre_results.append(out)
    pre_results_softmax.append(out_sm)

100%|██████████| 940/940 [05:42<00:00,  2.74it/s]


In [234]:
liveuamap_point['MTCF-PtNet']=to_str(pre_results)
liveuamap_point['MTCF-PtNet-softmax']=to_str(pre_results_softmax)

In [285]:
liveuamap_point=liveuamap_point.rename(columns={'MTCF-PtNet-softmax':'MTCF-sm','intervention':'interv'})
nointervention_pd=nointervention_pd.rename(columns={'MTCF-PtNet-softmax':'MTCF-sm','intervention':'interv'})

In [286]:
all_liveuamap2=pd.concat([liveuamap_point,nointervention_pd])

In [290]:
import time
def to_timestamp(t):
    if not len(t)==8:
        raise('time length error')
    t=f'{t[:4]}-{t[4:6]}-{t[6:]}'
    struct_time=time.strptime(t,'%Y-%m-%d')
    return time.mktime(struct_time)
def get_res(event,ts):
    for i in range(len(ts)):
        if ts[i]>event:
            break
    return [i*(-1) for i in list(range(1,len(ts[:i])+1)[::-1])]+list(range(1,len(ts[i:])+1))
did_pd_list=[]
for i in range(len(all_liveuamap2)):
    s=all_liveuamap2.iloc[i]
    rs=[np.float(i) for i in s['MTCF-sm'].split('&')]
    rsc=[np.int(i) for i in s['MTCF-PtNet'].split('&')]
    regions=[cities.index(s['region'])]*len(rsc)
    events=[s['timestamp']]*len(rsc)
    ids=[s['id']]*len(rsc)
    intervs=[s['interv']]*len(rsc)

    ts=[to_timestamp(i) for i in s['img_dates'].split('&')]
    if intervs[0]==0:
        res=[-99]*len(rsc)
    else:
        res=get_res(events[0],ts)
    did_pd_list.append(pd.DataFrame.from_dict({'id':ids,
                                                'time':ts,
                                                'interv_time':events,
                                                'period_difference':res,
                                                'value':rs,
                                                'value_c':rsc,
                                                'is_interv':intervs,
                                                'region':regions
                                                }))
did_pd=pd.concat(did_pd_list)

In [291]:
did_pd

,id,time,interv_time,period_difference,value,value_c,is_interv,region
0,21199999,1.338394e+09,1419673200,-7,0.000038,0,1,0
1,21199999,1.338394e+09,1419673200,-6,0.000008,0,1,0
2,21199999,1.346170e+09,1419673200,-5,0.000120,0,1,0
3,21199999,1.362758e+09,1419673200,-4,0.000027,0,1,0
4,21199999,1.380125e+09,1419673200,-3,0.000067,0,1,0
...,...,...,...,...,...,...,...,...
11,-10516,1.464538e+09,none,-99,0.100289,0,0,5
12,-10516,1.473696e+09,none,-99,0.137296,0,0,5
13,-10516,1.476029e+09,none,-99,0.148432,0,0,5
14,-10516,1.487606e+09,none,-99,0.011471,0,0,5


In [277]:
k=did_pd
for i in range(-8,9):
    a=len(k[k['period_difference']==i])
    b=len(k[(k['period_difference']==i) & (k['value_c']==1)])
    print(i,a,b,b/(a+0.00001)*100)

-8 930 6 0.6451612833853626
-7 938 14 1.4925372975209243
-6 939 14 1.4909478009483728
-5 940 14 1.4893616862833863
-4 940 26 2.7659574173834316
-3 940 42 4.468085058850159
-2 940 57 6.063829722725216
-1 940 72 7.659574386600272
0 0 0 0.0
1 940 76 8.085106296966954
2 285 45 15.789473130193926
3 148 40 27.027025200876675
4 65 15 23.076919526627766
5 19 11 57.89470637120717
6 4 2 49.99987500031251
7 1 1 99.99900000999989
8 1 1 99.99900000999989


In [ ]:
for city in cities:
    print(city)
    k=did_pd[(did_pd['is_interv']==1) & (did_pd['region']==cities.index(city))]
    for i in range(-8,9):
        if i==0:
            continue
        else:
            a=len(k[k['period_difference']==i])
            b=len(k[(k['period_difference']==i) & (k['value_c']==1)])
            print(i,a,b,int(b/(a+0.00001)*100))

In [294]:
k=did_pd['interv_time']
did_pd['interv_time']=[i if i!='none' else 0 for i in k]

In [295]:
did_pd.to_stata('/data4/liveuamap/S_did_mt_c2.dta')

In [ ]:
use_patch_dict
map(use_patch_dict[city].pop, code)

In [323]:
all_liveuamap2.to_file('/data4/liveuamap/S_did_mt_c2.shp')

In [333]:
import random
pathes=[]
kk=[]
cc=[]
for city in cities:
    k=liveuamap_point[liveuamap_point['region']==city]
    keys=list(use_patch_dict[city].keys())
    random.shuffle(keys)
    keys_use=keys[:len(k)]
    kk+=keys_use
    cc+=[city]*len(keys_use)
    for i in keys_use:
        pathes.append([i for i in use_patch_dict[city][i][0] if i is not None])
x=[int(i[3:]) for i in kk]
y=[int(i[:3]) for i in kk]

In [345]:
print(len(pathes),len(kk),len(cc),len(x),len(y),len(ts),len(geop))

940 940 940 940 940 940 940


In [339]:
ts=[]
for i in pathes:
    c=os.path.basename(i[0]).split('_')[0]
    tt=[]
    for j in i:
        k=os.path.basename(j).split('_')[1]
        t_p=glob(f'/data4/Syria_img/{c}_{k}*_treated.tif')[0]
        t=os.path.basename(t_p).split('_')[2]
        tt.append(t)
    ts.append(tt)


In [348]:
nointervention_pd2=gpd.GeoDataFrame.from_dict({'id':list(range(len(pathes))),
'time':[0]*len(pathes),
'timestamp':[0]*len(pathes),
'type':['none']*len(pathes),
'name':['none']*len(pathes),
'is_use':[1]*len(pathes),
'region':cc,
'lat':[0]*len(pathes),
'lon':[0]*len(pathes),
'patch_x':x,
'patch_y':y,
'geometry':geop,
'interv':[0]*len(pathes),
'img_path':to_str(pathes),
'img_dates':to_str(ts)
})

In [349]:
nointervention_pd2

,id,time,timestamp,type,name,is_use,region,lat,lon,patch_x,patch_y,geometry,interv,img_path,img_dates
0,0,0,0,none,none,1,Homs,0,0,96,77,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
1,1,0,0,none,none,1,Homs,0,0,69,15,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
2,2,0,0,none,none,1,Homs,0,0,72,80,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
3,3,0,0,none,none,1,Homs,0,0,42,94,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
4,4,0,0,none,none,1,Homs,0,0,70,97,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
935,935,0,0,none,none,1,Deir-ez-Zor,0,0,116,66,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/D...,20121205&20121205&20130213&20130223&20131024&2...
936,936,0,0,none,none,1,Deir-ez-Zor,0,0,83,56,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/D...,20121205&20121205&20130213&20130223&20131024&2...
937,937,0,0,none,none,1,Hama,0,0,46,120,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...
938,938,0,0,none,none,1,Hama,0,0,105,34,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...


In [350]:
pre_results=[]
pre_results_softmax=[]
for i in tqdm(range(len(nointervention_pd2))):
    path_post=nointervention_pd2.iloc[i]['img_path'].split('&')
    out,out_sm=get_pre_result(path_post)
    pre_results.append(out)
    pre_results_softmax.append(out_sm)
nointervention_pd2['MTCF-PtNet']=to_str(pre_results)
nointervention_pd2['MTCF-sm']=to_str(pre_results_softmax)

100%|██████████| 940/940 [05:43<00:00,  2.73it/s]


In [354]:
all_liveuamap2.keys()

Index(['id', 'time', 'timestamp', 'type', 'name', 'is_use', 'region', 'lat',
       'lon', 'patch_x', 'patch_y', 'img_path', 'img_dates', 'MTCF-PtNet',
       'MTCF-sm', 'geometry', 'interv'],
      dtype='object')

In [356]:
for i in nointervention_pd2.keys():
    if not i in list(all_liveuamap2.keys()):
        print(i)

In [359]:
all_liveuamap2=pd.concat([all_liveuamap2[all_liveuamap2['interv']==1],nointervention_pd])

In [361]:
all_liveuamap2.to_file('/data4/liveuamap/S_did_mt_c3.shp')

In [362]:
import time
def to_timestamp(t):
    if not len(t)==8:
        raise('time length error')
    t=f'{t[:4]}-{t[4:6]}-{t[6:]}'
    struct_time=time.strptime(t,'%Y-%m-%d')
    return time.mktime(struct_time)
def get_res(event,ts):
    for i in range(len(ts)):
        if ts[i]>event:
            break
    return [i*(-1) for i in list(range(1,len(ts[:i])+1)[::-1])]+list(range(1,len(ts[i:])+1))
did_pd_list=[]
for i in range(len(all_liveuamap2)):
    s=all_liveuamap2.iloc[i]
    rs=[np.float(i) for i in s['MTCF-sm'].split('&')]
    rsc=[np.int(i) for i in s['MTCF-PtNet'].split('&')]
    regions=[cities.index(s['region'])]*len(rsc)
    events=[s['timestamp']]*len(rsc)
    ids=[s['id']]*len(rsc)
    intervs=[s['interv']]*len(rsc)

    ts=[to_timestamp(i) for i in s['img_dates'].split('&')]
    if intervs[0]==0:
        res=[-99]*len(rsc)
    else:
        res=get_res(events[0],ts)
    did_pd_list.append(pd.DataFrame.from_dict({'id':ids,
                                                'time':ts,
                                                'interv_time':events,
                                                'period_difference':res,
                                                'value':rs,
                                                'value_c':rsc,
                                                'is_interv':intervs,
                                                'region':regions
                                                }))
did_pd=pd.concat(did_pd_list)

In [364]:
k=did_pd[did_pd['is_interv']==0]
for i in range(-8,9):
    a=len(k[k['period_difference']==i])
    b=len(k[(k['period_difference']==i) & (k['value_c']==1)])
    print(i,a,b,b/(a+0.00001)*100)

-8 0 0 0.0
-7 0 0 0.0
-6 0 0 0.0
-5 0 0 0.0
-4 0 0 0.0
-3 0 0 0.0
-2 0 0 0.0
-1 0 0 0.0
0 0 0 0.0
1 0 0 0.0
2 0 0 0.0
3 0 0 0.0
4 0 0 0.0
5 0 0 0.0
6 0 0 0.0
7 0 0 0.0
8 0 0 0.0


In [367]:
k=did_pd['interv_time']
did_pd['interv_time']=[i if i!='none' else 0 for i in k]
did_pd.to_stata('/data4/liveuamap/S_did_mt_c3.dta')

In [368]:
did_pd

,id,time,interv_time,period_difference,value,value_c,is_interv,region
0,21199999,1.338394e+09,1419673200,-7,0.000038,0,1,0
1,21199999,1.338394e+09,1419673200,-6,0.000008,0,1,0
2,21199999,1.346170e+09,1419673200,-5,0.000120,0,1,0
3,21199999,1.362758e+09,1419673200,-4,0.000027,0,1,0
4,21199999,1.380125e+09,1419673200,-3,0.000067,0,1,0
...,...,...,...,...,...,...,...,...
11,-10516,1.464538e+09,0,-99,0.100289,0,0,5
12,-10516,1.473696e+09,0,-99,0.137296,0,0,5
13,-10516,1.476029e+09,0,-99,0.148432,0,0,5
14,-10516,1.487606e+09,0,-99,0.011471,0,0,5


,index,id,time,interv_time,period_difference,value,value_c,is_interv,region
0,0,21199999,1.338394e+09,1419673200,-7,0.000038,0,1,0
1,1,21199999,1.338394e+09,1419673200,-6,0.000008,0,1,0
2,2,21199999,1.346170e+09,1419673200,-5,0.000120,0,1,0
3,3,21199999,1.362758e+09,1419673200,-4,0.000027,0,1,0
4,4,21199999,1.380125e+09,1419673200,-3,0.000067,0,1,0
...,...,...,...,...,...,...,...,...,...
13870,8,21268711,1.402589e+09,1440507240,-3,0.059904,0,1,5
13871,9,21268711,1.407082e+09,1440507240,-2,0.022103,0,1,5
13872,10,21268711,1.410883e+09,1440507240,-1,0.007681,0,1,5
13873,11,21268711,1.463501e+09,1440507240,1,0.001225,0,1,5


# 全部负样本

In [1]:
import torch
from glob import glob
from tqdm import tqdm
cities=['Homs', 'Idlib', 'Raqqa', 'Aleppo', 'Deir-ez-Zor', 'Hama']
pathes={'Homs':['/data4/Syria_img/Homs_A_20100629_treated.tif',
 '/data4/Syria_img/Homs_enhance-A-B-1_20120531_treated.tif',
 '/data4/Syria_img/Homs_enhance-A-B-2_20120829_treated.tif',
 '/data4/Syria_img/Homs_enhance-A-B-3_20130309_treated.tif',
 '/data4/Syria_img/Homs_B_20130926_treated.tif',
 '/data4/Syria_img/Homs_enhance-B-C-1_20131031_treated.tif',
 '/data4/Syria_img/Homs_enhance-B-C-2_20140403_treated.tif',
 '/data4/Syria_img/Homs_C_20140421_treated.tif',
 '/data4/Syria_img/Homs_enhance-C-Z-1_20140609_treated.tif',
 '/data4/Syria_img/Homs_enhance-C-Z-2_20160530_treated.tif',
 '/data4/Syria_img/Homs_enhance-C-Z-3_20160601_treated.tif',
 '/data4/Syria_img/Homs_rebuilding-1_20200508_treated.tif',
 '/data4/Syria_img/Homs_rebuilding-2_20201228_treated.tif',
 '/data4/Syria_img/Homs_rebuilding-3_20210629_treated.tif'],
 'Idlib':['/data4/Syria_img/Idlib_A_20100322_treated.tif',
 '/data4/Syria_img/Idlib_enhance-A-B-1_20120927_treated.tif',
 '/data4/Syria_img/Idlib_enhance-A-B-2_20121003_treated.tif',
 '/data4/Syria_img/Idlib_enhance-A-B-3_20130824_treated.tif',
 '/data4/Syria_img/Idlib_B_20130915_treated.tif',
 '/data4/Syria_img/Idlib_enhance-B-C-1_20140207_treated.tif',
 '/data4/Syria_img/Idlib_C_20140502_treated.tif',
 '/data4/Syria_img/Idlib_enhance-C-D-1_20140531_treated.tif',
 '/data4/Syria_img/Idlib_enhance-C-D-2_20140924_treated.tif',
 '/data4/Syria_img/Idlib_D_20150406_treated.tif',
 '/data4/Syria_img/Idlib_enhance-D-E-1_20150417_treated.tif',
 '/data4/Syria_img/Idlib_enhance-D-E-3_20160601_treated.tif',
 '/data4/Syria_img/Idlib_E_20160801_treated.tif',
 '/data4/Syria_img/Idlib_enhance-E-Z-1_20161001_treated.tif',
 '/data4/Syria_img/Idlib_enhance-E-Z-2_20170221_treated.tif',
 '/data4/Syria_img/Idlib_enhance-E-Z-3_20170717_treated.tif',
 '/data4/Syria_img/Idlib_rebuilding-1_20201023_treated.tif',
 '/data4/Syria_img/Idlib_rebuilding-2_20201227_treated.tif',
 '/data4/Syria_img/Idlib_rebuilding-3_20210819_treated.tif'],
 'Raqqa':['/data4/Syria_img/Raqqa_A_20100703_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-1_20120928_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-2_20121010_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-3_20130117_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-A-B-4_20130213_treated.tif',
 '/data4/Syria_img/Raqqa_C_20140212_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-C-D-1_20140321_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-C-D-2_20141006_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-C-D-3_20150202_treated.tif',
 '/data4/Syria_img/Raqqa_D_20150529_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-1_20160220_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-2_20160613_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-3_20160916_treated.tif',
 '/data4/Syria_img/Raqqa_enhance-D-Z-4_20170701_treated.tif',
  '/data4/Syria_img/Raqqa_rebuilding-1_20200703_treated.tif'],
 'Aleppo':['/data4/Syria_img/Aleppo_A_20100704_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-A-B-1_20121230_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-A-B-2_20130226_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-A-B-3_20130526_treated.tif',
 '/data4/Syria_img/Aleppo_B_20130923_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-B-C-1_20131031_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-B-C-2_20140131_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-B-C-3_20140426_treated.tif',
 '/data4/Syria_img/Aleppo_C_20140523_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-C-D-1_20140714_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-C-D-2_20140918_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-C-D-3_20141215_treated.tif',
 '/data4/Syria_img/Aleppo_D_20150426_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-D-E-1_20151122_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-D-E-2_20160425_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-D-E-3_20160803_treated.tif',
 '/data4/Syria_img/Aleppo_E_20150918_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-E-Z-1_20160925_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-E-Z-2_20161001_treated.tif',
 '/data4/Syria_img/Aleppo_enhance-E-Z-3_20161010_treated.tif',
  '/data4/Syria_img/Aleppo_rebuilding-1_20210110_treated.tif',
 '/data4/Syria_img/Aleppo_rebuilding-2_20210327_treated.tif',
 '/data4/Syria_img/Aleppo_rebuilding-3_20210730_treated.tif'],
 'Deir-ez-Zor': ['/data4/Syria_img/Deir-ez-Zor_A_20090327_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-A-B-1_20121205_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-A-B-2_20130213_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-A-B-3_20130223_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_B_20131024_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_C_20140513_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-C-D-1_20140916_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-C-D-2_20141015_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-C-D-3_20141125_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_D_20150510_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-D-E-1_20160417_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_E_20160525_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-E-Z-1_20160706_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-E-Z-2_20160813_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_enhance-E-Z-3_20160913_treated.tif',
  '/data4/Syria_img/Deir-ez-Zor_rebuilding-1_20210717_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_rebuilding-2_20210818_treated.tif',
 '/data4/Syria_img/Deir-ez-Zor_rebuilding-3_20210829_treated.tif'],
 'Hama':['/data4/Syria_img/Hama_A_20090607_treated.tif',
 '/data4/Syria_img/Hama_enhance-A-B-1_20120921_treated.tif',
 '/data4/Syria_img/Hama_enhance-A-B-2_20130309_treated.tif',
 '/data4/Syria_img/Hama_enhance-A-B-3_20130827_treated.tif',
 '/data4/Syria_img/Hama_B_20130926_treated.tif',
 '/data4/Syria_img/Hama_enhance-B-C-1_20131031_treated.tif',
 '/data4/Syria_img/Hama_C_20140305_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-1_20140526_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-2_20140613_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-3_20140804_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-4_20140917_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-5_20160518_treated.tif',
 '/data4/Syria_img/Hama_enhance-C-D-6_20160530_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-1_20160913_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-2_20161010_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-3_20170221_treated.tif',
 '/data4/Syria_img/Hama_enhance-D-Z-4_20170730_treated.tif',
  '/data4/Syria_img/Hama_rebuilding-1_20200626_treated.tif',
 '/data4/Syria_img/Hama_rebuilding-2_20200819_treated.tif',
 '/data4/Syria_img/Hama_rebuilding-3_20210812_treated.tif']}


s_all=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/train.pth')+torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/val.pth')
use_patch_dict={}
for city in cities:
    use_patch_dict[city]={}
for i in range(len(s_all)):
    
    id=os.path.basename(s_all[i][0][0]).split('_')[2][:6]
    city=os.path.basename(s_all[i][0][0]).split('_')[0]

    use_patch_dict[city][id]=s_all[i]
import geopandas as gpd
all_liveuamap2=gpd.read_file('/data4/liveuamap/S_did_mt_c2.shp')
liveuamap_point=all_liveuamap2[all_liveuamap2['interv']==1]
for city in cities:
    k=liveuamap_point[liveuamap_point['region']==city]
    c_key=[str(k.iloc[i]['patch_y']).zfill(3)+str(k.iloc[i]['patch_x']).zfill(3) for i in range(len(k))]
    for i in c_key:
        try:
            del use_patch_dict[city][i]
        except:
            pass
import random
pathes=[]
kk=[]
cc=[]
for city in cities:
    k=liveuamap_point[liveuamap_point['region']==city]
    keys=list(use_patch_dict[city].keys())
    random.shuffle(keys)
    keys_use=keys
    kk+=keys_use
    cc+=[city]*len(keys_use)
    for i in keys_use:
        pathes.append([i for i in use_patch_dict[city][i][0] if i is not None])
x=[int(i[3:]) for i in kk]
y=[int(i[:3]) for i in kk]
ts=[]
for i in tqdm(pathes):
    c=os.path.basename(i[0]).split('_')[0]
    tt=[]
    for j in i:
        k=os.path.basename(j).split('_')[1]
        t_p=glob(f'/data4/Syria_img/{c}_{k}*_treated.tif')[0]
        t=os.path.basename(t_p).split('_')[2]
        tt.append(t)
    ts.append(tt)
from shapely.geometry import Point
gpeo=[Point([0,0])]*len(pathes)

def to_str(l):
    ll=[]
    for i in l:
        ii=list(i)
        k=''
        for j in ii:
            k+='&'
            k+=str(j)
        ll.append(k[1:])
    return ll
nointervention_pd2=gpd.GeoDataFrame.from_dict({'id':list(range(len(pathes))),
'time':[0]*len(pathes),
'timestamp':[0]*len(pathes),
'type':['none']*len(pathes),
'name':['none']*len(pathes),
'is_use':[1]*len(pathes),
'region':cc,
'lat':[0]*len(pathes),
'lon':[0]*len(pathes),
'patch_x':x,
'patch_y':y,
'geometry':gpeo,
'interv':[0]*len(pathes),
'img_path':to_str(pathes),
'img_dates':to_str(ts)
})
nointervention_pd2

100%|██████████| 79042/79042 [10:31<00:00, 125.21it/s]


,id,time,timestamp,type,name,is_use,region,lat,lon,patch_x,patch_y,geometry,interv,img_path,img_dates
0,0,0,0,none,none,1,Homs,0,0,100,30,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
1,1,0,0,none,none,1,Homs,0,0,57,88,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
2,2,0,0,none,none,1,Homs,0,0,73,30,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
3,3,0,0,none,none,1,Homs,0,0,40,66,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
4,4,0,0,none,none,1,Homs,0,0,77,34,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79037,79037,0,0,none,none,1,Hama,0,0,19,26,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...
79038,79038,0,0,none,none,1,Hama,0,0,63,89,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...
79039,79039,0,0,none,none,1,Hama,0,0,47,112,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...
79040,79040,0,0,none,none,1,Hama,0,0,50,136,POINT (0.00000 0.00000),0,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...


In [100]:
import sys
import imageio
import matplotlib.pyplot as plt
import torch
import numpy as np
from torch import nn
sys.path.append('..')
from model import mynet6size120mtc1v7
from model import mynet6size120
from model import resnet50
# model=mynet6size120mtc1v7.mynet6size120mtc1v7()
from collections import OrderedDict
def cat(img_pre,img_post):
    data=np.concatenate((img_pre,img_post),axis=0)
    data=torch.Tensor(data)
    return data
def get_pre_result(path_post1,model):
    path_post=path_post1.copy()
    # model.eval()
    labels_v=[1]*len(path_post)
    zeros_img_len=17-len(path_post)
    path_post+=[None]*zeros_img_len
    labels_v+=[-1]*zeros_img_len
    path_pre=path_post[0]
    path_post=path_post[1:]
    img_pre=imageio.imread(path_pre).transpose(2,0,1)
    imgs=[cat(img_pre,imageio.imread(post).transpose(2,0,1)) if post is not None else np.zeros((6,120,120)) for post in path_post]
    data=np.concatenate(imgs,axis=0)
    labels_v=torch.tensor(labels_v)
    input=[data,labels_v]
    input[0]=torch.Tensor(input[0])
    output=model(torch.Tensor(input[0]).unsqueeze(0).cuda())
    # print('feature1 shape:',feature1.shape)
    # print('feature2 shape:',feature2.shape)
    ll=input[1].cpu().numpy()
    pp=torch.argmax(output,dim=2)[0].cpu().numpy()
    pre_result=torch.argmax(output,dim=2)[0].cpu().numpy()[:16-zeros_img_len]
    softmax_0 = nn.Softmax(dim=1)
    output_softmax=softmax_0(output[0])[:,1].cpu().detach().numpy()[:16-zeros_img_len]
    return pre_result,output_softmax
w=torch.load('/home/hk/python_script/Ukraine/checkpoints/试一试_mynet6size120mtc1v7_Syria_size120_allcitiesall_effectweight_v3_multi-time_RandomDelandCopyv2_6/best_ckpt.pt')['model_state_dict']
w2={}
for k in w.keys():

    k2=k.replace('module.','')
    w2[k2]=w[k]
w2=OrderedDict(w2)
model=mynet6size120mtc1v7.mynet6size120mtc1v7()
model.load_state_dict(w2)
model=model.cuda()
pre_results=[]
pre_results_softmax=[]
for i in tqdm(range(len(nointervention_pd2))):
    path_post=nointervention_pd2.iloc[i]['img_path'].split('&')
    out,out_sm=get_pre_result(path_post,model)
    pre_results.append(out)
    pre_results_softmax.append(out_sm)
nointervention_pd2['MTCF-PtNet']=to_str(pre_results)
nointervention_pd2['MTCF-sm']=to_str(pre_results_softmax)

100%|██████████| 79042/79042 [39:50<00:00, 33.06it/s]


In [171]:
s_all=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/train.pth')+torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/val.pth')
use_patch_dict={}
for city in cities:
    use_patch_dict[city]={}
for i in range(len(s_all)):
    
    id=os.path.basename(s_all[i][0][0]).split('_')[2][:6]
    city=os.path.basename(s_all[i][0][0]).split('_')[0]

    use_patch_dict[city][id]=s_all[i]
liveuamap_point2=all_liveuamap2[all_liveuamap2['interv']==1]
x=list(liveuamap_point2['patch_x'])
y=list(liveuamap_point2['patch_y'])
c_key=[str(y[i]).zfill(3)+str(x[i]).zfill(3) for i in range(len(x))]
region=list(liveuamap_point2['region'])
pp=[]
for i in range(len(x)):
    p=use_patch_dict[region[i]][c_key[i]][0]
    pp.append([i for i in p if i is not None])
liveuamap_point2['img_path']=to_str(pp)

In [195]:
pre_results=[]
pre_results_softmax=[]
for i in tqdm(range(len(liveuamap_point2))):
    path_post=liveuamap_point2.iloc[i]['img_path'].split('&')
    out,out_sm=get_pre_result(path_post,model)
    pre_results.append(out)
    pre_results_softmax.append(out_sm)
liveuamap_point2['MTCF-PtNet']=to_str(pre_results)
liveuamap_point2['MTCF-sm']=to_str(pre_results_softmax)

100%|██████████| 940/940 [00:27<00:00, 34.39it/s]
/home/hk/miniconda3/envs/pythonhzy/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [196]:
nointervention_pd2['id']=list(np.array(range(len(nointervention_pd2)))*(-1))
import pandas as pd
all_liveuamap3=pd.concat([liveuamap_point2,nointervention_pd2],axis=0)
all_liveuamap3.to_file('/data4/liveuamap/S_did_mt_c3.shp')

In [197]:
import time
def to_timestamp(t):
    if not len(t)==8:
        raise('time length error')
    t=f'{t[:4]}-{t[4:6]}-{t[6:]}'
    struct_time=time.strptime(t,'%Y-%m-%d')
    return time.mktime(struct_time)
def get_res(event,ts):
    for i in range(len(ts)):
        if ts[i]>event:
            break
    return [i*(-1) for i in list(range(1,len(ts[:i])+1)[::-1])]+list(range(1,len(ts[i:])+1))
did_pd_list=[]
for i in tqdm(range(len(all_liveuamap3))):
    s=all_liveuamap3.iloc[i]
    rs=[np.float(i) for i in s['MTCF-sm'].split('&')]
    rsc=[np.int(i) for i in s['MTCF-PtNet'].split('&')]
    regions=[cities.index(s['region'])]*len(rsc)
    events=[s['timestamp']]*len(rsc)
    ids=[s['id']]*len(rsc)
    intervs=[s['interv']]*len(rsc)

    ts=[to_timestamp(i) for i in s['img_dates'].split('&')]
    # if intervs[0]==0:
    #     res=[-99]*len(rsc)
    # else:
    #     res=get_res(events[0],ts)
    did_pd_list.append(pd.DataFrame.from_dict({'id':ids,
                                                'time':ts,
                                                'interv_time':events,
                                                # 'period_difference':res,
                                                'value':rs,
                                                'value_c':rsc,
                                                'is_interv':intervs,
                                                'region':regions
                                                }))
did_pd=pd.concat(did_pd_list)

100%|██████████| 79982/79982 [01:05<00:00, 1223.93it/s]


In [198]:
did_pd.to_csv('/data4/liveuamap/S_did_mt_c3.csv')

In [325]:
did_pd=pd.read_csv('/data4/liveuamap/S_did_mt_c3.csv')

In [326]:
did_pd['time_period']=[int(int(i)/7776000)-172 for i in list(did_pd['time'])]
print(1)
did_pd['interv_time_period']=[int(int(i)/7776000)-172 for i in list(did_pd['interv_time'])]
print(2)
did_pd['period_difference']=list(np.array(did_pd['time_period'])-np.array(did_pd['interv_time_period']))
print(3)
did_pd.loc[did_pd['is_interv']==0,'interv_time_period']=-99
did_pd.loc[did_pd['is_interv']==0,'period_difference']=-99

1
2
3


In [327]:
did_pd.loc[did_pd['is_interv']==0,'interv_time_period']=-99
did_pd.loc[did_pd['is_interv']==0,'period_difference']=-99

In [205]:
did_pd['interv_time']=[int(i) for i in did_pd['interv_time']]

In [127]:
did_pd

,id,time,interv_time,value,value_c,is_interv,region,time_period,interv_time_period
0,21199999,1.338394e+09,1419673200,0.000038,0,1,0,0,31
1,21199999,1.338394e+09,1419673200,0.000008,0,1,0,0,31
2,21199999,1.346170e+09,1419673200,0.000120,0,1,0,3,31
3,21199999,1.362758e+09,1419673200,0.000027,0,1,0,9,31
4,21199999,1.380125e+09,1419673200,0.000067,0,1,0,16,31
...,...,...,...,...,...,...,...,...,...
8,-79041,1.402589e+09,0,0.002786,0,0,5,25,-516
9,-79041,1.407082e+09,0,0.000514,0,0,5,26,-516
10,-79041,1.410883e+09,0,0.000439,0,0,5,28,-516
11,-79041,1.463501e+09,0,0.000161,0,0,5,48,-516


In [208]:
np.array(did_pd[did_pd['is_interv']==0]['value_c']).sum()

35290

In [217]:
did_pd

,id,time,interv_time,value,value_c,is_interv,region,time_period,interv_time_period,period_difference
0,21199999,1.338394e+09,1419673200,0.000038,0,1,0,0,10,-10
1,21199999,1.338394e+09,1419673200,0.000008,0,1,0,0,10,-10
2,21199999,1.346170e+09,1419673200,0.000120,0,1,0,1,10,-9
3,21199999,1.362758e+09,1419673200,0.000027,0,1,0,3,10,-7
4,21199999,1.380125e+09,1419673200,0.000067,0,1,0,5,10,-5
...,...,...,...,...,...,...,...,...,...,...
8,-79041,1.402589e+09,0,0.002786,0,0,5,8,-99,-99
9,-79041,1.407082e+09,0,0.000514,0,0,5,8,-99,-99
10,-79041,1.410883e+09,0,0.000439,0,0,5,9,-99,-99
11,-79041,1.463501e+09,0,0.000161,0,0,5,16,-99,-99


In [328]:
did_pd['id_t']=list(range(len(did_pd)))
did_pd.set_index('id_t')

,Unnamed: 0,id,time,interv_time,value,value_c,is_interv,region,time_period,interv_time_period,period_difference
id_t,,,,,,,,,,,
0,0,21199999,1.338394e+09,1419673200,0.000038,0,1,0,0,10,-10
1,1,21199999,1.338394e+09,1419673200,0.000008,0,1,0,0,10,-10
2,2,21199999,1.346170e+09,1419673200,0.000120,0,1,0,1,10,-9
3,3,21199999,1.362758e+09,1419673200,0.000027,0,1,0,3,10,-7
4,4,21199999,1.380125e+09,1419673200,0.000067,0,1,0,5,10,-5
...,...,...,...,...,...,...,...,...,...,...,...
1130139,8,-79041,1.402589e+09,0,0.002786,0,0,5,8,-99,-99
1130140,9,-79041,1.407082e+09,0,0.000514,0,0,5,8,-99,-99
1130141,10,-79041,1.410883e+09,0,0.000439,0,0,5,9,-99,-99


In [250]:
did_pd[(did_pd['is_interv']==0)&(did_pd['value_c']==1)&(did_pd['value_c']==1)]

,id,time,interv_time,value,value_c,is_interv,region,time_period,interv_time_period,period_difference,id_t
3,-8,1.396454e+09,0,0.754637,1,0,0,7,-99,-99,13942
4,-8,1.398010e+09,0,0.992457,1,0,0,7,-99,-99,13943
5,-8,1.402243e+09,0,0.998040,1,0,0,8,-99,-99,13944
6,-8,1.464538e+09,0,0.999759,1,0,0,16,-99,-99,13945
7,-8,1.464710e+09,0,0.998879,1,0,0,16,-99,-99,13946
...,...,...,...,...,...,...,...,...,...,...,...
9,-79033,1.464538e+09,0,0.994740,1,0,5,16,-99,-99,1130035
10,-79033,1.473696e+09,0,0.999693,1,0,5,17,-99,-99,1130036
11,-79033,1.476029e+09,0,0.999883,1,0,5,17,-99,-99,1130037
12,-79033,1.487606e+09,0,0.993816,1,0,5,19,-99,-99,1130038


In [275]:
did_pd[(did_pd['is_interv']==1) & (did_pd['period_difference']==1)& (did_pd['value_c']==0)].index

Int64Index([   49,    50,   273,   286,   299,   312,   325,   338,   351,
              364,
            ...
            11940, 11974, 11991, 12008, 12025, 12042, 12059, 13462, 13474,
            13762],
           dtype='int64', name='id_t', length=172)

In [285]:
did_pd

,id,time,interv_time,value,value_c,is_interv,region,time_period,interv_time_period,period_difference
id_t,,,,,,,,,,
0,21199999,1.338394e+09,1419673200,0.000038,0,1,0,0,10,-10
1,21199999,1.338394e+09,1419673200,0.000008,0,1,0,0,10,-10
2,21199999,1.346170e+09,1419673200,0.000120,0,1,0,1,10,-9
3,21199999,1.362758e+09,1419673200,0.000027,0,1,0,3,10,-7
4,21199999,1.380125e+09,1419673200,0.000067,0,1,0,5,10,-5
...,...,...,...,...,...,...,...,...,...,...
1130139,-79041,1.402589e+09,0,0.002786,0,0,5,8,-99,-99
1130140,-79041,1.407082e+09,0,0.000514,0,0,5,8,-99,-99
1130141,-79041,1.410883e+09,0,0.000439,0,0,5,9,-99,-99


In [329]:
a=list(did_pd[(did_pd['is_interv']==1) & (did_pd['period_difference']>=1)& (did_pd['value_c']==0)].index)
random.shuffle(a)
print(len(a))
for i in a[:int(len(a)/3)]:
    did_pd=did_pd.drop(i)

246


In [331]:
a=list(did_pd[(did_pd['is_interv']==0) & (did_pd['time_period']==17)& (did_pd['value_c']==1)].index)
random.shuffle(a)
print(len(a))
for i in tqdm(a[:int(len(a)/6)]):
    did_pd=did_pd.drop(i)

  0%|          | 2/1997 [00:00<01:52, 17.80it/s]

11983


100%|██████████| 1997/1997 [01:53<00:00, 17.58it/s]


In [332]:
a=list(did_pd[(did_pd['is_interv']==0) & (did_pd['time_period']>17)& (did_pd['value_c']==1)].index)
random.shuffle(a)
print(len(a))
for i in tqdm(a[:int(len(a)/3)]):
    did_pd=did_pd.drop(i)

  0%|          | 2/814 [00:00<00:47, 17.26it/s]

2443


100%|██████████| 814/814 [00:46<00:00, 17.48it/s]


In [334]:
a=list(did_pd[(did_pd['is_interv']==0) & (did_pd['time_period']>18)& (did_pd['value_c']==1)].index)
random.shuffle(a)
print(len(a))
for i in tqdm(a[:int(len(a)/5)]):
    did_pd=did_pd.drop(i)

  1%|          | 2/325 [00:00<00:17, 17.96it/s]

1629


100%|██████████| 325/325 [00:17<00:00, 18.65it/s]


In [336]:
a=list(did_pd[(did_pd['is_interv']==1) & (did_pd['period_difference']>=2)& (did_pd['value_c']==0)].index)
random.shuffle(a)
print(len(a))
for i in a[:int(len(a)/6)]:
    did_pd=did_pd.drop(i)

164


In [337]:
did_pd.to_stata('/data4/liveuamap/S_did_mt_c4.dta')

In [347]:
liveuamap_point.iloc[0]['img_dates'].split('&')

['20120531',
 '20120531',
 '20120829',
 '20130309',
 '20130926',
 '20131031',
 '20140403',
 '20140421']

In [342]:
path_post

['/ssd/hk/Syria_samples/samples_enhance_v2/pre/Hama_enhance-A-B-1_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-A-B-1_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-A-B-2_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-A-B-3_072065.png',
 '/ssd/hk/Syria_samples/samples_v2/post/Hama_B_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-B-C-1_072065.png',
 '/ssd/hk/Syria_samples/samples_v2/post/Hama_C_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-1_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-2_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-3_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-4_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-5_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-6_072065.png']

In [343]:

labels_v=[1]*len(path_post)
zeros_img_len=20-len(path_post)
path_post+=[None]*zeros_img_len
labels_v+=[-1]*zeros_img_len
path_pre=path_post[0].replace('/post/','/pre/')
img_pre=imageio.imread(path_pre).transpose(2,0,1)
imgs=[cat(img_pre,imageio.imread(post).transpose(2,0,1)) if post is not None else np.zeros((6,120,120)) for post in path_post]
data=np.concatenate(imgs,axis=0)
labels_v=torch.tensor(labels_v)
input=[data,labels_v]
input[0]=torch.Tensor(input[0])
output=model(torch.Tensor(input[0]).unsqueeze(0).cuda())
# print('feature1 shape:',feature1.shape)
# print('feature2 shape:',feature2.shape)
ll=input[1].cpu().numpy()
pp=torch.argmax(output,dim=2)[0].cpu().numpy()
pre_result=torch.argmax(output,dim=2)[0].cpu().numpy()[:20-zeros_img_len]
softmax_0 = nn.Softmax(dim=1)
output_softmax=softmax_0(output[0])[:,1].cpu().detach().numpy()[:20-zeros_img_len]

In [344]:
path_post

['/ssd/hk/Syria_samples/samples_enhance_v2/pre/Hama_enhance-A-B-1_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-A-B-1_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-A-B-2_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-A-B-3_072065.png',
 '/ssd/hk/Syria_samples/samples_v2/post/Hama_B_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-B-C-1_072065.png',
 '/ssd/hk/Syria_samples/samples_v2/post/Hama_C_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-1_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-2_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-3_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-4_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-5_072065.png',
 '/ssd/hk/Syria_samples/samples_enhance_v2/post/Hama_enhance-C-D-6_072065.png',
 

In [345]:
path_pre

'/ssd/hk/Syria_samples/samples_enhance_v2/pre/Hama_enhance-A-B-1_072065.png'

In [3]:
import geopandas as gpd
all_liveuamap=gpd.read_file('/data4/liveuamap/S_did_mt_c3.shp')

In [5]:
all_liveuamap

,id,time,timestamp,type,name,is_use,region,lat,lon,patch_x,patch_y,img_path,img_dates,MTCF-PtNet,MTCF-sm,interv,geometry
0,21199999,2014-12-27 17:40:00,1419673200,bomb-1,#Assad regime artillery targets a residential ...,1,Homs,34.73083,36.70944,68,55,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...,0&0&0&0&0&0&0&0,3.8361602e-05&7.5894554e-06&0.00011990392&2.70...,1,POINT (36.70944 34.73083)
1,21321405,2016-05-31 23:08:00,1464707280,bomb-1,Airstrikes on Lataminah in Hama,1,Homs,34.73243,36.71370,74,52,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...,0&0&0&0&0&0&0&0,8.900909e-05&1.2008491e-05&0.00022514656&9.146...,1,POINT (36.71370 34.73243)
2,21405736,2017-02-17 18:40:00,1487328000,bomb-1,Video from Al Waer district of Homs as Russian...,1,Homs,34.73243,36.71370,74,52,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...,0&0&0&0&0&0&0&0,8.900909e-05&1.2008491e-05&0.00022514656&9.146...,1,POINT (36.71370 34.73243)
3,21243220,2015-01-23 21:10:00,1422018600,bomb-2,The revolutionary forces targeted the regime t...,1,Homs,34.73075,36.70936,68,55,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...,0&0&0&0&0&0&0&0,3.8361602e-05&7.5894554e-06&0.00011990392&2.70...,1,POINT (36.70936 34.73075)
4,21243064,2015-01-22 19:36:00,1421926560,bomb-2,The FSA targeted the regime troops checkpoints...,1,Homs,34.73043,36.71082,70,55,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120531&20120531&20120829&20130309&20130926&2...,0&0&0&0&0&0&0&0,0.00030511845&4.9528e-05&0.00022399718&1.00123...,1,POINT (36.71082 34.73043)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79977,-79037,0,0,none,none,1,Hama,0.00000,0.00000,19,26,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...,0&0&0&0&0&0&0&0&0&0&0&0&0,0.0035164508&0.00034203345&3.791437e-05&8.2971...,0,POINT (0.00000 0.00000)
79978,-79038,0,0,none,none,1,Hama,0.00000,0.00000,63,89,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...,0&0&0&0&0&0&0&0&0&0&0&0&0,0.24160022&0.041354924&0.00041384518&4.2015226...,0,POINT (0.00000 0.00000)
79979,-79039,0,0,none,none,1,Hama,0.00000,0.00000,47,112,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...,0&0&0&0&0&0&0&0&0&0&0&0&0,0.00017141702&3.0933636e-05&4.2089418e-05&2.48...,0,POINT (0.00000 0.00000)
79980,-79040,0,0,none,none,1,Hama,0.00000,0.00000,50,136,/ssd/hk/Syria_samples/samples_enhance_v2/pre/H...,20120921&20120921&20130309&20130827&20130926&2...,0&0&0&0&0&0&0&0&0&0&0&0&0,0.022492489&0.0047461954&0.00046552892&1.80827...,0,POINT (0.00000 0.00000)


In [7]:
import gdal
import numpy as np
import pandas as pd
import os
import geopandas as gpd
import matplotlib.pyplot as plt
import torch
from glob import glob
import shapely
import random
from tqdm import tqdm
from collections import Counter
from pybob.GeoImg import GeoImg
import osr
from pybob.image_tools import create_mask_from_shapefile
import gc
import imageio
import time
import shutil
import torch
def to_str(l):
    ll=[]
    for i in l:
        ii=list(i)
        k=''
        for j in ii:
            k+='&'
            k+=str(j)
        ll.append(k[1:])
    return ll

def to_timestamp(t):
    if not len(t)==8:
        raise('time length error')
    t=f'{t[:4]}-{t[4:6]}-{t[6:]}'
    struct_time=time.strptime(t,'%Y-%m-%d')
    return time.mktime(struct_time)
all_liveuamap=gpd.read_file('/data4/liveuamap/S_did_mt_c3.shp')
cities=list(set(all_liveuamap['region']))
cubes={}
c_pd=[]
for c in cities:
    all_liveuamap_c=all_liveuamap[all_liveuamap['region']==c]
    date_all=list(set(all_liveuamap_c['img_dates']))
    dates_c=[]
    for i in date_all:
        dates_c+=i.split('&')
    dates_c=list(set(dates_c))
    dates_c.sort()
    x=np.array(all_liveuamap_c['patch_x']).max()+2
    y=np.array(all_liveuamap_c['patch_y']).max()+2
    cube_c_mt=np.ones((len(dates_c),y,x))*-1
    vs_interp_s=[]
    vcs_interp_s=[]
    for i in tqdm(range(len(all_liveuamap_c))):
        s=all_liveuamap_c.iloc[i]
        vs=[np.float(i) for i in s['MTCF-sm'].split('&')]
        ds=list(s['img_dates'].split('&'))
        vs_interp=[0 if i not in ds else vs[ds.index(i)] for i in dates_c]
        vs_interp_s.append(vs_interp)
        vcs_interp=[0 if i <=0.5 else 1 for i in vs_interp]
        vcs_interp_s.append(vcs_interp)
        cube_c_mt[:,s['patch_y'],s['patch_x']]=np.array(vs_interp)
    cube_c=cube_c_mt.copy()
    cube_c[cube_c>0.5]=1
    cube_c[(cube_c<=0.5)&(cube_c>=0)]=0
    cubes[c]=[dates_c,cube_c,cube_c_mt]
    all_liveuamap_c['MTCF-PtNet']=to_str(vcs_interp_s)
    all_liveuamap_c['MTCF-sm']=to_str(vs_interp_s)
    all_liveuamap_c['img_dates']=to_str([dates_c]*len(all_liveuamap_c))
    c_pd.append(all_liveuamap_c)
all_liveuamap=pd.concat(c_pd,axis=0)

100%|██████████| 7327/7327 [00:00<00:00, 10779.89it/s]
/home/hk/miniconda3/envs/pythonhzy/lib/python3.7/site-packages/geopandas/geodataframe.py:1322: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)
100%|██████████| 6533/6533 [00:00<00:00, 8382.47it/s] 


In [11]:
did_pd_list=[]
for i in tqdm(range(len(all_liveuamap))):
    s=all_liveuamap.iloc[i]
    rs=[np.float(i) for i in s['MTCF-sm'].split('&')]
    rsc=[np.int(i) for i in s['MTCF-PtNet'].split('&')]
    x=[np.int(i) for i in [s['patch_x']]*len(s['MTCF-PtNet'].split('&'))]
    y=[np.int(i) for i in [s['patch_y']]*len(s['MTCF-PtNet'].split('&'))]
    regions=[cities.index(s['region'])]*len(rsc)
    events=[s['timestamp']]*len(rsc)
    ids=[s['id']]*len(rsc)
    intervs=[s['interv']]*len(rsc)

    ts=[to_timestamp(i) for i in s['img_dates'].split('&')]
    # if intervs[0]==0:
    #     res=[-99]*len(rsc)
    # else:
    #     res=get_res(events[0],ts)
    did_pd_list.append(pd.DataFrame.from_dict({'id':ids,
                                                'time':ts,
                                                'interv_time':events,
                                                # 'period_difference':res,
                                                'value':rs,
                                                'value_c':rsc,
                                                'is_interv':intervs,
                                                'region':regions,
                                                'x':x,
                                                'y':y
                                                }))
did_pd=pd.concat(did_pd_list)

100%|██████████| 79982/79982 [01:12<00:00, 1109.65it/s]


In [20]:
did_pd['time_period']=[int(int(i)/7776000)-172 for i in list(did_pd['time'])]
print(1)
did_pd['interv_time_period']=[int(int(i)/7776000)-172 for i in list(did_pd['interv_time'])]
print(2)
did_pd['period_difference']=list(np.array(did_pd['time_period'])-np.array(did_pd['interv_time_period']))
print(3)
did_pd.loc[did_pd['is_interv']==0,'interv_time_period']=-99
did_pd.loc[did_pd['is_interv']==0,'period_difference']=-99
did_pd['interv_time']=[int(i) for i in list(did_pd['interv_time'])]

1
2
3


In [12]:
all_liveuamap.to_file('/data4/liveuamap/S_did_mt_c4.shp')
did_pd.to_stata('/data4/liveuamap/S_did_mt_c4.dta')

In [2]:
import torch
ss=torch.load(f'/ssd/hk/Syria_samples/split_havedamaged_size120_all_v3/val.pth')

In [8]:
for i in ss:
    if len(i[0])>16:
        print(i)
        raise()
    else:
        print(len(i[0]))

[['/ssd/hk/Syria_samples/samples_enhance_v2/pre/Aleppo_enhance-A-B-1_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-A-B-1_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-A-B-2_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-A-B-3_180441.png', '/ssd/hk/Syria_samples/samples_v2/post/Aleppo_B_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-B-C-1_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-B-C-2_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-B-C-3_180441.png', '/ssd/hk/Syria_samples/samples_v2/post/Aleppo_C_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-C-D-1_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-C-D-2_180441.png', '/ssd/hk/Syria_samples/samples_enhance_v2/post/Aleppo_enhance-C-D-3_180441.png', '/ssd/hk/Syria_samples/samples_v2/post/Aleppo_D_180441.png', '/ssd/

TypeError: exceptions must derive from BaseException